In [2]:
import os
import io
import datetime
import logging
import sys

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 

import civis
import civis.io
from civis.futures import CivisFuture

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, RandomForestRegressor
from civis.ml import ModelPipeline

import uuid
import json
from pprint import pprint
import tempfile
import concurrent.futures
from concurrent.futures import wait
from collections import namedtuple

In [3]:
# Default feature lists for Rainbow Modeling Frame (each number corresponds to number of features)
feature_table = civis.io.read_civis_sql(sql='select * from bernie_nmarchio2.feature_list order by sort_order asc', use_pandas = True, database='Bernie 2020')
feature_list_large = list(feature_table[(feature_table['frame_large'] == 1)]['feature_name']) + ['state_code']
feature_list_medium = list(feature_table[(feature_table['frame_medium'] == 1)]['feature_name']) + ['state_code']
feature_list_small = list(feature_table[(feature_table['frame_small'] == 1)]['feature_name']) + ['state_code']

additional_features = ['attempt_count', 'rank_1_is_cell_phone',
                       'num_phones_1', 'num_phones_2', 'num_phones_3', 'num_phones_other']

feature_list_large.extend(additional_features)
feature_list_medium.extend(additional_features)
feature_list_small.extend(additional_features)

table_columns = civis.io.read_civis_sql(
    sql=f'''select ordinal_position as position, column_name, data_type 
    from information_schema.columns 
    where table_name = 'rainbow_modeling_frame' and table_schema = 'bernie_data_commons' and column_name != 'person_id'
    order by ordinal_position;''', use_pandas = True, database='Bernie 2020')

#exclusion_list_466 = [e for e in list(table_columns['column_name']) if e not in feature_list_466] 

In [4]:
len(feature_list_small)

205

In [10]:
# USER INPUT CELL

# DV table parameters
DATABASE = 'Bernie 2020'
# Primary key in both the DV table and the Modeling Frame
PRIMARY_KEY = 'person_id' 
# Table containing recoded Dependent Variables keyed to the PRIMARY_KEY
DV_TABLE = 'bernie_cherdeman.contactibility_outcomes'
# List of binarized dependent variables (accepts 1, 0, and null values) in DV_TABLE
DV_LIST = [ 'pickup_first', 'pickup_last', 'pickup_ever', 
            'id_first','id_last', 'id_ever'] # 'pickup_first', 'id_first',


# Modeling frame table parameters
# Table containing covariates and keyed to PRIMARY_KEY
MODELING_FRAME = 'bernie_cherdeman.contactibility_modeling_frame'
# Columns in the Modeling Frame to exclude from feature list (i.e., strings or incomplete coverage)
EXCLUSION_COLUMNS = ['jsonid','state_code','census_block_group_2010', 'person_id', 'us_region', 
                     'pickup_first', 'pickup_last', 'pickup_ever', 'id_first', 'id_last', 'id_ever']

# Output table parameters
# Schema to contain prediction tables
SCHEMA = 'bernie_cherdeman'
# String that will be concatenated in front of the output table's name
PREFIX = 'getthru'

# Sampling parameters
# Non-response training data
    # True: automatically select people not in DV_TABLE at random from Phoenix (assumes person_id is PRIMARY_KEY)
    # False: automatically select people where the DV equals 0 from the DV_TABLE
SAMPLE_FROM_PHOENIX = False
# Number of non-response classes per target class (default is 2) 
CLASS_BALANCE = 2
# Maximum number of targets to randomly sample from DV_TABLE
MAX_TARGET_COUNT = 40000

In [6]:
datestamp = '{:%Y%m%d}'.format(datetime.date.today())

In [7]:
# Counts of positive classes
dv_sql_targets = "\n,".join(["sum({dv}) as {dv}".format(dv=i) for i in DV_LIST])
sql_collapse_targets = f"""select {dv_sql_targets} from {DV_TABLE};"""
sql_count_targets = civis.io.read_civis_sql(sql_collapse_targets, DATABASE)


In [8]:
sql_count_targets

[['pickup_first',
  'pickup_last',
  'pickup_ever',
  'pickup_last',
  'id_last',
  'id_ever'],
 ['202624', '236411', '341585', '236411', '257413', '337340']]

In [ ]:
# Determing training table proportion of positives to negatives (to avoid class imbalance problems)
# sample_share = []
# for i in range(len(DV_LIST)):
#     if int(sql_count_targets[1][i]) > MAX_TARGET_COUNT:
#         sql_count_targets[1][i] = MAX_TARGET_COUNT
#     u = round(int(sql_count_targets[1][i])*CLASS_BALANCE)
#     sample_share.append(u)      

In [ ]:
# for i in range(len(DV_LIST)):
#     dv_item = DV_LIST[i]
#     random_sample = sample_share[i]
#     if SAMPLE_FROM_PHOENIX is True:
#         zero_sample = f'''(select p.person_id, 0 as {dv_item} from phoenix_analytics.person p left join (select person_id from {DV_TABLE}) d on p.person_id = d.person_id where d.person_id is null and is_deceased = false and reg_record_merged = false and reg_on_current_file = true and reg_voter_flag = true order by random() limit {random_sample})'''   
#     if SAMPLE_FROM_PHOENIX is False:
#         zero_sample = f'''(select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 order by random() limit {random_sample})'''


In [14]:
# Create training tables
#for i, dv_item in enumerate(DV_LIST):
#     if (int(sql_count_targets[1][i])*3) <= 1000:
    #feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_small])
#     if (int(sql_count_targets[1][i])*3) > 1000 & (int(sql_count_targets[1][i])*3) <= 2000:
         
    #feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_medium])
#     if (int(sql_count_targets[1][i])*3) > 2000:
    #feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_large])
    #dv_item = DV_LIST[i]
#print(dv_item)
#     random_sample = sample_share[i]
#     if SAMPLE_FROM_PHOENIX is True:
#         zero_sample = f'''(select p.person_id, 0 as {dv_item} from phoenix_analytics.person p left join (select person_id from {DV_TABLE}) d on p.person_id = d.person_id where d.person_id is null and is_deceased = false and reg_record_merged = false and reg_on_current_file = true and reg_voter_flag = true order by random() limit {random_sample})'''   
#     if SAMPLE_FROM_PHOENIX is False:
#         zero_sample = f'''(select {PRIMARY_KEY}, {dv_item} from {DV_TABLE} where {dv_item} = 0 order by random() limit {random_sample})'''
feature_select = "\n,".join(["{feature}".format(feature=f) for f in feature_list_small])
dvs = "\n,".join(["{dv}".format(dv=d) for d in DV_LIST])

training_sql1 = f"""DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_training_0 CASCADE;
                       CREATE TABLE {SCHEMA}.{PREFIX}_training_0 DISTKEY({PRIMARY_KEY}) SORTKEY({PRIMARY_KEY}) AS (
                           select * 
                           from (select {PRIMARY_KEY}, {dvs} 
                               from {DV_TABLE}
                               where stratification < 2500)
                           join (select {PRIMARY_KEY}, {feature_select}
                                 from {MODELING_FRAME})
                           using({PRIMARY_KEY})
                           );"""

training_sql2 = f"""DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_training_1 CASCADE;
                       CREATE TABLE {SCHEMA}.{PREFIX}_training_1 DISTKEY({PRIMARY_KEY}) SORTKEY({PRIMARY_KEY}) AS (
                           select * 
                           from (select {PRIMARY_KEY}, {dvs} 
                               from {DV_TABLE}
                               where stratification between 2500 and 5000)
                           join (select {PRIMARY_KEY}, {feature_select}
                                 from {MODELING_FRAME})
                           using({PRIMARY_KEY})
                           );"""
training = [training_sql1, training_sql2]

for t in training:
    create_training_sql = civis.io.query_civis(t, database=DATABASE)
    create_training_sql.result().state
    print("one down")
    

    
"""select * 
   from (select {PRIMARY_KEY}, {cd} 
       from {DV_TABLE}
       where {dv_item}_strat < 5000)
   join (select {PRIMARY_KEY}, {feature_select}
         from {MODELING_FRAME})
   using({PRIMARY_KEY});"""

"""select * 
   from {MODELING_FRAME}
   where pickup_strat < 2001"""
    

one down
one down


'select * \n   from {MODELING_FRAME}\n   where pickup_strat < 2001'

In [22]:
#combine training sets into view
training_sql = f"""drop view if exists {SCHEMA}.{PREFIX}_training;
                   create view {SCHEMA}.{PREFIX}_training as (
                        (select * from {SCHEMA}.{PREFIX}_training_0)
                        union
                        (select * from {SCHEMA}.{PREFIX}_training_1)
                    );
                   """
combined_training_sql = civis.io.query_civis(training_sql, database=DATABASE)
combined_training_sql.result().state

'succeeded'

In [23]:
# Train models
train_list = []
model_list = []

for i, dv in enumerate(DV_LIST):
    print('TRAINING >>> {}'.format(dv))
    
    exc_list = DV_LIST.copy()
    exc_list.remove(dv)
    
    assert dv not in exc_list 
    
    for m in ['random_forest_classifier', 'sparse_logistic']:
    
        name = f"""{dv}_{m}_{datestamp}"""
        model = ModelPipeline(model=m,
                              dependent_variable=dv,
                              primary_key=PRIMARY_KEY,
                              excluded_columns=EXCLUSION_COLUMNS,
                              #calibration='sigmoid',
                              model_name=name,
                              memory_requested=15000#,
                              #disk_requested=5
                             )
    
        where_string = '{} is not null'.format(dv)
        # Use 
        train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training""", 
                            database_name=DATABASE,
                            sql_where=where_string#,
                            #fit_params={'sample_weight': WEIGHT_VAR}
                           )

        model_list.append(model)
        train_list.append(train) 

name = f"""multioutcome_rf_{datestamp}"""
model = ModelPipeline(model='random_forest_classifier',
                      dependent_variable=DV_LIST,
                      primary_key=PRIMARY_KEY,
                      excluded_columns=EXCLUSION_COLUMNS,
                      #calibration='sigmoid',
                      model_name=name,
                      memory_requested=15000#,
                      #disk_requested=5
                     )

train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training""", 
                    database_name=DATABASE,
                    sql_where='''where pickup_first is not null 
                                 and pickup_last is not null
                                 and pickup_ever is not null
                                 and id_first is not null
                                 and id_last is not null
                                 and id_ever is not null'''#,
                    #fit_params={'sample_weight': WEIGHT_VAR}
                   )

model_list.append(model)
train_list.append(train)


TRAINING >>> pickup_first
TRAINING >>> pickup_last
TRAINING >>> pickup_ever
TRAINING >>> id_first
TRAINING >>> id_last
TRAINING >>> id_ever


In [33]:
name = f"""multioutcome_rf_{datestamp}_v4"""
model = ModelPipeline(model='random_forest_classifier',
                      dependent_variable=DV_LIST,
                      primary_key=PRIMARY_KEY,
                      excluded_columns=EXCLUSION_COLUMNS,
                      #calibration='sigmoid',
                      model_name=name,
                      memory_requested=15000#,
                      #disk_requested=5
                     )

train = model.train(table_name=f"""{SCHEMA}.{PREFIX}_training""", 
                    database_name=DATABASE,
                    sql_where='''pickup_first is not null 
                                 and pickup_last is not null
                                 and pickup_ever is not null
                                 and id_first is not null
                                 and id_last is not null
                                 and id_ever is not null'''#,
                    #fit_params={'sample_weight': WEIGHT_VAR}
                   )

model_list.append(model)
train_list.append(train)

In [34]:
len(train_list)

16

In [37]:
# Extract successful models
model_output = model_list
train_output = train_list

jobs_list = []
for t in train_output: 
    try:
        if len(t.metadata['output']) > 0:  
            jobs_list.append(t)
            print('Job success')
    except:
        print('Job failure')
        pass

    
model_output, train_output = zip(*((m, t) for m, t in zip(model_output, train_output) if t in jobs_list))
model_output = list(model_output)
train_output = list(train_output)

Job success
Job success
Job success
Job success
Job success
Job success
Job success
Job success
Job success
Job success
Job success
Job success
Job failure
Job failure
Job failure
Job success


In [42]:
print(len(jobs_list))
print(len(model_output))

13
13


In [44]:
# Generate validation metrics
metrics_list = []

for b in train_output:
    if b.job_id != 59761951:
        metric = {'job_id':b.job_id,
                  'run_id':b.run_id,
                  'dv': ''.join(b.metadata['run']['configuration']['data']['y']),
                  'model': b.metadata['model']['model'],
                  'time_of_train_run': b.metadata['run']['time_of_run'],
                  'n_rows': b.metadata['data']['n_rows'],
                  'n_features': b.metadata['data']['n_cols'],
                  'auc': b.metadata['metrics']['roc_auc'],
                  'deciles': b.metadata['metrics']['deciles'],
                  'confusion_matrix': b.metadata['metrics']['confusion_matrix'],
                  'accuracy': b.metadata['metrics']['accuracy'],
                  'p_correct': b.metadata['metrics']['p_correct'],
                  'pop_incidence_true': b.metadata['metrics']['pop_incidence_true'],
                  'feature_list':b.metadata['model']['parameters']['relvars']
                 }
        metrics_list.append(metric)
    
metric_order = (['job_id', 'run_id', 'dv', 'model', 'time_of_train_run', 'n_rows', 'n_features',
                 'auc', 'deciles', 'confusion_matrix', 'accuracy', 'p_correct','pop_incidence_true','feature_list'])

validation_df = pd.DataFrame.from_records(metrics_list, columns=metric_order, index='run_id')
validation_df

,job_id,dv,model,time_of_train_run,n_rows,n_features,auc,deciles,confusion_matrix,accuracy,p_correct,pop_incidence_true,feature_list
run_id,,,,,,,,,,,,,
206047980,59759537,pickup_first,random_forest_classifier,2020-01-30T16:22:35Z,68036,204,0.769170,"[0.07304526748971193, 0.11215640158753491, 0.1...","[[46731, 581], [19211, 1513]]",0.709095,"[0.9877198173824823, 0.07300714147847906]","[0.6953965547651243, 0.30460344523487565]","[civis_2020_marriage, civis_2020_children_pres..."
206047993,59759544,pickup_first,sparse_logistic,2020-01-30T16:22:02Z,68036,204,0.757209,"[0.08597883597883597, 0.10157283551374394, 0.1...","[[42357, 4955], [12859, 7865]]",0.738168,"[0.8952696990192763, 0.37951167728237795]","[0.6953965547651243, 0.30460344523487565]","[civis_2020_children_present, civis_2020_spani..."
206047997,59759550,pickup_last,random_forest_classifier,2020-01-30T16:22:42Z,68036,204,0.752649,"[0.09082892416225749, 0.1306776422166691, 0.18...","[[42671, 1037], [21519, 2809]]",0.668470,"[0.9762743662487416, 0.11546366326866163]","[0.6424245987418425, 0.35757540125815745]","[civis_2020_marriage, civis_2020_children_pres..."
206048012,59759567,pickup_last,sparse_logistic,2020-01-30T16:22:07Z,68036,204,0.745124,"[0.09141681363903587, 0.12803175069822137, 0.1...","[[36374, 7334], [12840, 11488]]",0.703481,"[0.8322046307312162, 0.47221308780006577]","[0.6424245987418425, 0.35757540125815745]","[civis_2020_marriage, civis_2020_children_pres..."
206048016,59759573,pickup_ever,random_forest_classifier,2020-01-30T16:22:54Z,68036,204,0.752540,"[0.12595532039976484, 0.17683374981625755, 0.2...","[[27258, 9040], [12019, 19719]]",0.690473,"[0.7509504655903907, 0.6213056903396559]","[0.5335116702921983, 0.46648832970780174]","[civis_2020_marriage, civis_2020_children_pres..."
206048031,59759587,pickup_ever,sparse_logistic,2020-01-30T16:21:52Z,68036,204,0.738766,"[0.12007642563198119, 0.18903424959576656, 0.2...","[[24378, 11920], [9587, 22151]]",0.683888,"[0.6716072510882142, 0.6979330770684983]","[0.5335116702921983, 0.46648832970780174]","[civis_2020_marriage, civis_2020_children_pres..."
206048041,59759594,id_first,random_forest_classifier,2020-01-30T16:22:52Z,68036,204,0.780054,"[0.033362727807172254, 0.05100690871674261, 0....","[[54041, 0], [13995, 0]]",0.794300,"[1.0, 0.0]","[0.7943000764301252, 0.20569992356987477]","[civis_2020_marriage, civis_2020_children_pres..."
206048053,59759599,id_first,sparse_logistic,2020-01-30T16:21:45Z,68036,204,0.750056,"[0.04659024103468548, 0.05541672791415552, 0.0...","[[53501, 540], [13357, 638]]",0.795740,"[0.99000758683222, 0.04558770989639157]","[0.7943000764301252, 0.20569992356987477]","[civis_2020_marriage, civis_2020_children_pres..."
206048061,59759609,id_last,random_forest_classifier,2020-01-30T16:22:59Z,68036,204,0.737336,"[0.058495002939447385, 0.08937233573423489, 0....","[[50007, 168], [17478, 383]]",0.740637,"[0.9966517189835575, 0.021443368232461787]","[0.7374772179434417, 0.26252278205655827]","[civis_2020_marriage, civis_2020_children_pres..."


In [45]:
# Write validation metrics to Redshift
create_validation_table = civis.io.dataframe_to_civis(df=validation_df,
                                                 database=DATABASE, 
                                                 table= f'{SCHEMA}.{PREFIX}_validation_{datestamp}', 
                                                 existing_table_rows='drop')


In [ ]:
# Score the voterfile
scores_list = []
for m,t in zip(model_output, train_output):
    DV_NAME = ''.join(t.metadata['run']['configuration']['data']['y'])
    print(DV_NAME)
    SCORES_TABLE = f'{SCHEMA}.{PREFIX}_{DV_NAME}_{datestamp}'
    scores_list.append(SCORES_TABLE)
    scores = m.predict(primary_key=PRIMARY_KEY,
                       database_name=DATABASE, 
                       table_name=MODELING_FRAME,
                       if_exists='drop',
                       output_table=SCORES_TABLE,
                       disk_space=10)
scores.result()


In [ ]:
# Generate SQL for final output table and drop intermediary tables
input_train_list = []
output_score_list = []
for i in range(len(DV_LIST)):
    input_train = f"{SCHEMA}.{PREFIX}_training_{i}"
    input_train_list.append(input_train)
    output_score = f"{SCHEMA}.{PREFIX}_{DV_LIST[i]}_{datestamp}"
    output_score_list.append(output_score)

drop_input_train_sql = "\n".join(["drop table if exists {tbl};".format(tbl=v) for v in input_train_list])
drop_output_score_sql = "\n".join(["drop table if exists {tbl};".format(tbl=t) for t in output_score_list])  
dv_strings = "\n,".join(["{dv_score}_1 as {dv_score}".format(dv_score=dv) for dv in DV_LIST])
dv_tiles = "\n,".join(["NTILE(100) OVER (ORDER BY {dv_tile}_1) AS {dv_tile}_100".format(dv_tile=dv) for dv in DV_LIST])
join_table = []
if len(output_score_list) > 1:
    for i in output_score_list[1:]:
        j = str(' left join '+f'{i}'+f' using({PRIMARY_KEY}) ')
        join_table.append(j)
        #dv_strings = "\nleft join ".join(["{dv_score}".format(table=tbl) for tbl in table_list[i])


In [ ]:
output_table_sql = f"""
set query_group to 'importers';
set wlm_query_slot_count to 3;
DROP TABLE IF EXISTS {SCHEMA}.{PREFIX}_output_{datestamp};
CREATE TABLE {SCHEMA}.{PREFIX}_output_{datestamp}
  DISTSTYLE KEY
  DISTKEY ({PRIMARY_KEY})
  SORTKEY ({PRIMARY_KEY})
  AS ("""+'select '+ f"{PRIMARY_KEY} \n," + dv_strings + "\n," + dv_tiles + ' from '+ ''.join(output_score_list[0]) + ''.join(join_table) +');'  


In [ ]:
print(output_table_sql)

In [ ]:
# Create final output table
create_output_table = civis.io.query_civis(sql=output_table_sql, database=DATABASE)
create_output_table.result().state


In [ ]:
# Drop intermediary tables
drop_input_train_query = civis.io.query_civis(sql=drop_input_train_sql, database=DATABASE)
drop_input_train_query.result().state

drop_output_score_query = civis.io.query_civis(sql=drop_output_score_sql, database=DATABASE)
drop_output_score_query.result().state

In [ ]:
print(drop_input_train_sql)
print(drop_output_score_sql)

In [ ]:
# Grant team on tables
grant_statement = f"""
GRANT ALL ON SCHEMA {SCHEMA} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_output_{datestamp} TO GROUP bernie_data;
GRANT SELECT ON {SCHEMA}.{PREFIX}_validation_{datestamp} TO GROUP bernie_data;
"""
grant_team = civis.io.query_civis(sql=grant_statement, database=DATABASE)
grant_team.result().state

In [ ]:
print(grant_statement)